In [1]:
import cobra
import csv
import pandas as pd

In [2]:
high_gene_set = set()
with open("high_genes.csv",mode='r') as f:
    handle = csv.reader(f,delimiter=',')
    lines = list(handle)
for line in lines[1:]:
    high_gene_set.add(line[1].rstrip())

In [3]:
common_rxn_set = set()
with open("common_clinical.txt",mode='r') as f:
    for line in f:
        common_rxn_set.add(line.rstrip())

In [4]:
model = cobra.io.read_sbml_model("iYL1228.xml")

In [5]:
rxn_set = set()
gene_set = set()
source_node = []
target_node = []


In [ ]:
#create network dataframe
for x in high_gene_set:
    if x not in gene_set:
        try:
            gene = model.genes.get_by_id(x)
        except:
            pass
        else:
            rxns_of_gene = [rxn.id for rxn in gene.reactions]
            for y in rxns_of_gene:
                if not y in rxn_set:
                    try:
                        rxn = model.reactions.get_by_id(y)
                    except:
                        pass
                    else:
                        genes_of_rxn = [gene.id for gene in rxn.genes]
                        for z in genes_of_rxn:
                            source_node.append(y)
                            target_node.append(model.genes.get_by_id(z).name)
                            rxn_set.add(y)
                            gene_set.add(z)

In [7]:
for x in common_rxn_set:
    if x not in rxn_set:
        try:
            rxn = model.reactions.get_by_id(x)
        except:
            pass
        else:
            genes_of_rxn = [gene.id for gene in rxn.genes]
            for y in genes_of_rxn:
                if not y in gene_set:
                    try:
                        gene = model.genes.get_by_id(y)
                    except:
                        pass
                    else:
                        rxns_of_gene = [rxn.id for rxn in gene.reactions]
                        for z in rxns_of_gene:
                            source_node.append(model.genes.get_by_id(y).name)
                            target_node.append(z)
                            gene_set.add(y)
                            rxn_set.add(z)

In [8]:
network_dict = {
    'source_node' : source_node,
    'target_node' : target_node 
}
df_network = pd.DataFrame(network_dict)
df_network.to_csv("common-high_network2.csv",index=False)

In [ ]:
#create nodes dataframe
nodes = []
colour = []
shape = []
outline = []

In [157]:
for rxn in rxn_set:
    #nodes
    nodes.append(rxn)
    #colour
    compartment_set = model.reactions.get_by_id(rxn).compartments
    compartment_list = []
    for x in compartment_set:
        compartment_list.append(x)
        compartment_list = sorted(compartment_list)
    compartment_str = "".join(compartment_list)
    colour.append(compartment_str)
    #shape
    shape.append("rect")
    #outline
    if rxn in common_rxn_set:
        outline.append("bold")
    else:
        outline.append("no")


In [158]:
for gene in gene_set:
    try:
        #nodes
        nodes.append(model.genes.get_by_id(gene).name)
    except:
        pass
    else:
        #colour
        colour.append("gene")
        #shape
        shape.append("circ")
        #outline
        if gene in high_gene_set:
            outline.append("bold")
        else:
            outline.append("no")

In [159]:
nodes_dict = {
    'nodes' : nodes,
    'colour' : colour,
    'shape' : shape,
    'outline' : outline
}
df_nodes = pd.DataFrame(nodes_dict)
df_nodes.to_csv("common-high_nodes2.csv",index=False)

In [142]:
count = 0
for gene in gene_set:
    try:
        #nodes
        nodes.append(model.genes.get_by_id(gene).name)
    except:
        pass
    else:
        if gene in high_gene_set:
            count += 1

print(count)

32


In [9]:
source_node_set = set(source_node)
target_node_set = set(target_node)
intersect = source_node_set.intersection(target_node_set)
print(intersect)

set()


In [23]:
#nodes dataframe v2
nodes = []
node_type = []
is_high = []
is_common = []
is_vf = []
compartment = []

vf_set = {
    "fimK","fimH","fimG","fimF","fimD","fimC","fimI","fimA","fimE","fimB",
    "mkrJ","mrkI","mrkH","rcsB","galF","iroD","sciN","tssJ","tssG","tssF",
    "hcp","tssD","clpV","tssH","vgrG","ompA","dotU","tssL","vasE","tssK",
    "vipB","tssC","vipA","tssB","entA","entB","entE","entC","entC","entS",
    "entF","fes","fepA","entD","iutA","fepB","fepD","fepG","fepC","allD",
    "allC","allB","allR","allA","allS","acrA","acrB"
}


In [24]:
for rxn in rxn_set:
    nodes.append(rxn)
    node_type.append("rxn")
    is_high.append("NA")
    if rxn in common_rxn_set:
        is_common.append("Y")
    else:
        is_common.append("N")
    is_vf.append("NA")
    #ccompartment
    compartment_set = model.reactions.get_by_id(rxn).compartments
    compartment_list = []
    for x in compartment_set:
        compartment_list.append(x)
        compartment_list = sorted(compartment_list)
    compartment_str = "".join(compartment_list)
    compartment.append(compartment_str)
    

In [25]:
for gene in gene_set:
    try:
        gene_name = model.genes.get_by_id(gene).name
    except:
        pass
    else:
        nodes.append(gene_name)
        node_type.append("gene")
        if gene in high_gene_set:
            is_high.append("Y")
        else:
            is_high.append("N")
        is_common.append("NA")
        if gene in vf_set:
            is_vf.append("Y")
        else:
            is_vf.append("N")
        compartment.append("NA")

In [27]:
nodes_dict = {
    'nodes' : nodes,
    'node_type' : node_type,
    'is_high' : is_high,
    'is_vf' : is_vf,
    'is_common' : is_common,
    'compartment' : compartment
}
df_nodes = pd.DataFrame(nodes_dict)
df_nodes.to_csv("common-high_nodes3.csv",index=False)